In [1]:
import pandas as pd

In [2]:

file_path=r'F:\Sharry\Masterlist\Master_List'

In [3]:
df=pd.read_csv(file_path)

In [4]:
df.head()

,Serial No.,SC Decision,Project Title,Scientist Organization,Concept Proposal Paper Discipline / Sector Involved,Concept Proposal Duration,Concept Proposal Paper Proposed Solution,Objectives,Methodology/Activities,Outcomes (Quantifiable),Estimated Budget (in millions)
0,2,Recommended,Energy Storage and Trading for Smart Grids: Le...,National Skills University,Artificial intelligence,2.0,Products,1) The proposed system seeks to meet the needs...,"Data collection, AI model development, device ...",The quantifiable results of an AI-based device...,9.0
1,3,Recommended,Embedded AI Based Real Time Circuit Breaker He...,National Skills University,Artificial intelligence,2.0,Technologies,1)The suggested embedded AI-based real-time he...,Stages of this project's methodology:\n1. Syst...,Quantifiable results\nThe integrated AI-based ...,9.0
2,4,Recommended,Smart Farming: An Intelligent Crop Recommendat...,University of Agriculture (FSD),Artificial intelligence,2.0,Value-Addition,•To build an intelligent device to help farmer...,Data collection: The device would collect data...,I.Increased crop yield: A crop recommendation ...,7.0
3,5,Recommended,Banking Guard: Counterfeit Fraud Using ML,"Superior University, Lahore",Artificial intelligence,1.0,Products,•Developing and deploying a fraud detection sy...,Some common activities performed in a project ...,•Reduced financial losses: By detecting and pr...,3.0
4,6,Recommended,AI-Driven Predictive Maintenance of Induction...,Pakistan Institute of Nuclear Science &amp; Te...,Artificial intelligence,2.0,Techniques,•To demonstrate the feasibility of artificial ...,For the successful implementation of the propo...,•minimize maintenance costs by 25%\n•approxima...,1.5


In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

# Combine textual columns for embedding
textual_columns = [
    "Project Title",
    "Objectives",
    "Methodology/Activities",
    "Outcomes (Quantifiable)",
    "Concept Proposal Paper Discipline / Sector Involved",
]
df['combined_text'] = df[textual_columns].fillna('').agg(' '.join, axis=1)

In [6]:
from langchain_ollama import OllamaEmbeddings
# Generate embeddings for combined textual data
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

In [7]:
from langchain.schema import Document

# Wrap each row of text in a Document object
documents = [
    Document(page_content=row) for row in df['combined_text'].tolist()
]


In [9]:
from langchain.vectorstores import FAISS

# Create the FAISS index using the embedding model
vector_store = FAISS.from_documents(documents=documents, embedding=embedding_model)


In [21]:
vector_store.search(query = """
Establishing a drone-based system to monitor and prevent illegal deforestation in protected forest reserves. 
The system integrates satellite data and AI to provide real-time alerts to authorities.


""",search_type="mmr")

[Document(metadata={}, page_content='Exploring Artificial Intelligence and Geospatial Techniques for the Quantification of Forest Fires  in Margalla Hills National Park •Quantification of forest fires and CO2 emissions in the study area i.e. Margallah Hills National Parks through remote sensing techniques \n•Spatial identification of forest fires hotspots in the study area using multiples spectral indices including the burnt area mapping\n•Identification of potential forest fires in the study area using multi-criteria AHP method\n•Development and design of an early forest fire detection system using Artificial Intelligence and Unmanned Aerial Vehicles/Drone technology\n The study aimed to develop an early detection/ warning system of forest fires to optimize fire control and management in wild and protected areas. This research study involves an integrated approach and extensive data collection both primary and secondary including the satellite imagery, ancillary data, field data, vari

In [13]:
from langchain_ollama import OllamaLLM

# Initialize the Ollama LLM
llm = OllamaLLM(model="llama3.1")  


In [14]:
evaluator_prompt = """ 
You are an expert evaluator for research grant applications. Your task is to analyze the provided project proposal and compare it with similar projects from a dataset. For each of the aspects outlined below, you will need to provide:

1. A similarity score (0% to 100%) indicating how closely the project matches existing projects in the dataset.
2. A detailed analysis of whether the project is unique or redundant, with a focus on what differentiates it or aligns it with existing work.
3. If the project is unique, highlight specific innovative or novel aspects that make it stand out and explain how these features enhance its chances of receiving a grant.
4. If the project is highly similar to existing ones, provide a brief explanation of how this redundancy may affect the likelihood of grant approval, and suggest areas for improvement to reduce overlap and increase uniqueness.

Aspects to analyze:
- **Project Title**: Does the title reflect innovation or does it echo similar projects? Is the title clear and impactful for potential stakeholders?
- **Objectives**: Are the objectives clearly defined and realistic? How do the objectives differentiate this project from others in terms of ambition or relevance?
- **Methodology/Activities**: Assess whether the methodology is novel and grounded in strong scientific principles, or if it closely mirrors existing methods. Is there a unique approach to solving the problem?
- **Outcomes (Quantifiable)**: How measurable and impactful are the expected outcomes? Are these outcomes aligned with key grant criteria, such as societal relevance, scalability, and feasibility? How does this compare to similar projects?
- **Discipline/Sector Involved**: Does the project address an underserved sector or a new discipline? Does it bring a fresh perspective or contribution to a well-established field?

Additionally, you will need to:
- **Generate a structured summary** of similarity scores for each aspect, formatted for easy visualization (e.g., as a table or in a structured report).
- **Grant Recommendation**: Provide a final recommendation on whether the project is suitable for grant approval based on your analysis of its uniqueness and similarity. Include any actionable feedback on improving the project's chances of approval, especially in terms of enhancing its uniqueness or addressing any gaps identified.

Input Project:
{query}

Similar Projects:
{similar_projects}

Expected Output:
1. A detailed similarity report for each aspect, including:
   - Similarity percentage score (0% to 100%) for each aspect.
   - In-depth analysis of uniqueness or redundancy for each aspect.
   - Clear differentiation of innovative features if the project is unique.
2. A summary table of similarity scores for easy graph generation (include categories like "Project Title," "Objectives," etc.).
3. A final evaluation and recommendation for the grant decision, based on the overall uniqueness and relevance of the project. Include any suggestions for improving the project if it is too similar to existing ones.
"""


In [15]:
student_prompt = """
You are an assistant for evaluating the likelihood of grant approval for a given project. Assess the project's similarity to previous projects in the dataset and calculate the likelihood of grant approval. Provide a detailed and balanced evaluation based on the following considerations:
1. Higher similarity with previous projects reduces the likelihood of grant approval.
2. Unique and innovative features significantly increase the chances of grant approval (up to 80-90% if the project is highly unique).
3. Consider how well the project addresses unmet needs or gaps in the dataset.

Input Details:
- Input Project: {query}
- Similar Projects: {similar_projects}

Your task:
1. Analyze the similarity of the project to previous projects and describe how it impacts the chances of approval.
2. Highlight any innovative or unique aspects that may positively influence the decision. If the project is deemed highly unique, assign a grant likelihood score of 80-90%.
3. Provide actionable suggestions for further improvements to enhance the project's chances of grant approval.
4. Address areas where the project could reduce redundancy or better align with unmet needs in the dataset.

Output:
- A clear and concise likelihood evaluation in percentages.
- A brief explanation of factors influencing the score, including both positive (e.g., uniqueness) and negative (e.g., redundancy) aspects.
- Actionable suggestions to improve the project for resubmission, focusing on enhancing innovation and addressing gaps.
- For highly unique projects, explicitly state why the likelihood score is 80-90%, supported by detailed reasoning.
"""

In [23]:
# Simulated project input
query = """
Establishing a drone-based system to monitor and prevent illegal deforestation in protected forest reserves. 
The system integrates satellite data and AI to provide real-time alerts to authorities.



"""

# Query embedding
query_embedding = embedding_model.embed_query(query)

# Retrieve similar projects
results = vector_store.similarity_search_by_vector(query_embedding, k=3)
similar_projects = "\n".join([res.page_content for res in results])

# Generate evaluator's report
evaluator_report = llm.invoke(
    evaluator_prompt.format(query=query, similar_projects=similar_projects)
)

# Generate student's likelihood report
student_report = llm.invoke(
    student_prompt.format(query=query, similar_projects=similar_projects)
)

# Output results
print("Evaluator's Perspective:")
print(evaluator_report)
print("\n" + "=" * 50 + "\n")
print("Student's Perspective:")
print(student_report)


Evaluator's Perspective:
**Similarity Report**

### Project Title

* Similarity score: 60%
* Analysis: The title "AI-powered System for Detecting and Preventing Crop Diseases in Wheat" is somewhat unique, but it shares similarities with other projects that focus on AI applications in agriculture. However, the specific emphasis on wheat crops differentiates it from more general agricultural projects.
* Differentiation: This project's title focuses on a specific crop (wheat), which sets it apart from broader agricultural projects.

### Objectives

* Similarity score: 80%
* Analysis: The objectives of this project are similar to those in other agriculture-related AI projects. They involve using AI for disease detection, reducing costs, and improving agricultural practices.
* Differentiation: While the overall goal is not unique, the specific approach (focusing on wheat crops) and detailed objectives provide some differentiation.

### Methodology

* Similarity score: 70%
* Analysis: The pr